In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [106]:
df=pd.read_csv('cleaned_Titanic.csv')
df

,Survived,Pclass,Age,Fare,Family,Sex_male,Embarked_Q,Embarked_S,Deck_B,Deck_C,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,0,3,22.0,1.906724,1,1,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1,1,38.0,3.497640,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,1,3,26.0,1.970459,0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
3,1,1,35.0,3.304258,1,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
4,0,3,35.0,1.981680,0,1,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,2.326029,0,1,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
887,1,1,19.0,2.916885,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
888,0,3,28.0,2.745246,3,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
889,1,1,26.0,2.916885,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


**X & y**

In [109]:
X=df.drop('Survived',axis=1)
y=df['Survived']

**Scaling**

In [112]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X.iloc[:,[1,2]]=ss.fit_transform(X.iloc[:,[1,2]])

**PCA**

In [115]:
# from sklearn.decomposition import PCA
# pca=PCA(n_components=0.95)
# X=pca.fit_transform(X)

In [117]:
X=pd.DataFrame(X)

In [119]:
X.shape

(891, 30)

**Hyperparameter tuning**

In [122]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state = True)
model.fit(X, y)

DecisionTreeClassifier(random_state=True)

In [124]:
model.get_depth()

20

In [126]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(random_state = True)
param_grid = {'criterion' : ['gini', 'entropy'], 'max_depth' : list(range(1, 21))}

gs = GridSearchCV(estimator, param_grid, cv = 5, scoring = 'accuracy')
gs.fit(X, y)
gs.best_params_

{'criterion': 'entropy', 'max_depth': 4}

**Modelling + Evaluation**

In [129]:
#Modelling
model = gs.best_estimator_
model

DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=True)

**Identifying imp columns**

In [132]:
f = pd.DataFrame(model.feature_importances_, columns = ['Feature Importance'], index = X.columns.tolist())
f

,Feature Importance
Pclass,0.131903
Age,0.041387
Fare,0.099127
Family,0.104174
Sex_male,0.077301
Embarked_Q,0.000000
Embarked_S,0.000000
Deck_B,0.000000
Deck_C,0.000000
Deck_D,0.000000


In [134]:
imp = f[f['Feature Importance'] > 0].index.tolist()
imp

['Pclass', 'Age', 'Fare', 'Family', 'Sex_male', 'Title_Master', 'Title_Mr']

In [136]:
X = X.loc[:, imp]

**Final Model**

In [139]:
#modeling
model=gs.best_estimator_
model.fit(X,y)

#prediction
ypred=model.predict(X)

#Evaluation 
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y,ypred)

from sklearn.model_selection import cross_val_score
CV=cross_val_score(model,X,y,cv=5,scoring='accuracy').mean()

print('accuracy:',accuracy)
print('CV score:', CV)

if (abs(accuracy - CV)<=0.05):
    print('Good Model')
else:
    print('Bad Model')

accuracy: 0.8462401795735129
CV score: 0.8316364321134895
Good Model


**Save the best model**

In [65]:
# from joblib import dump
# dump(model,'titanic_model.joblib')